In [1]:
import numpy as np
import qubits
import bell_state

Send message base on pure state (classic prototype)

In [2]:
#Alice and Bob know state_ab, Alice gets qubit_a, Bob has qubit_b
state_ab = "00"

#Alice set message in qubit_c state (0 or 1)
message = 1
qubit_c = qubits.get_qubit_matrix([message])

#Alice do bell measurement on qubit_c and qubit_a to build entangled relationship
#and this operation change the state of qubit_b base on qubit_c at the same time
#then Alice send measurement result in traditional communication method to Bob
qubit_b,state_ca = bell_state.teleportation(qubit_c,state_ab)

#Bob use gates on qubit_b base on the measurement result
#how to choose gates: 
#    measurement result is base on entangled relationship between qubit_c and qubit_a which is same like the relationship between qubit_a and qubit_b
#    which means that how the measurement result changed from state_ab to state_bc is same like the state that qubit_c changed to qubit_b
#    so when Bob know the state_ab and state_ca, he could find the way to restore state_ab from state_ca
#    and he could do the same operation on qubit_b to restore the state of orignal qubit_c and get the message
qubit_b = bell_state.unitary_operation(qubit_b, 0, state_ca, state_ab)
state_b = qubits.measurement(qubit_b,[0])[0]
print("qubit_c(sender):{} | measurement_ca(send):{} | qubit_b(receiver):{}".format(message, state_ca, state_b))

qubit_c(sender):1 | measurement_ca(send):00 | qubit_b(receiver):1


Send message base on superposition state (message in probability)

In [3]:
batch_n = 10 ** 4
state_ab = "00"
message = 7
decode_key = 5
qubit_c = qubits.normalization(np.array([[message],
                                         [decode_key]]))
receive = {"0":0,"1":0}
print("encode_qubit_c:\n{}".format(qubit_c))

#receive different state_ca and measure qubit_b for many times, qubit_b is superposition state
qubit_b, state_ca = bell_state.teleportation(qubit_c, state_ab)
qubit_b = bell_state.unitary_operation(qubit_b, 0, state_ca, state_ab)
for i in range(batch_n):
    state_b = qubits.measurement(qubit_b, [0])[0]
    receive[state_b] = receive[state_b]+1
print(receive)
receive_message = decode_key/receive["1"]*receive["0"]
print("message(sender):{}(decimal) = {}(bianary bit) | batch of qubit:{} | decode_probability(receiver):{} => {}".format(message,len(bin(message).split("0b")[-1]),batch_n,receive_message,round(receive_message)))

encode_qubit_c:
[[0.81373347]
 [0.58123819]]
{'0': 5842, '1': 4158}
message(sender):7(decimal) = 3(bianary bit) | batch of qubit:10000 | decode_probability(receiver):7.025012025012026 => 7


Conclusion: use superposition state to send message would waste a lot of qubit which is a bad idea to send nomal infomation

Send message base on classic state (massage encode as 0 or 1)

In [4]:
message = "Hello World!"
print("[Send]:{}".format(message))
message_encode = format(int(bytes(message, 'utf-8').hex(), base=16), 'b')
print("[Encode]:{}".format(message_encode))

[Send]:Hello World!
[Encode]:10010000110010101101100011011000110111100100000010101110110111101110010011011000110010000100001


In [5]:
receive_buffer=""
for m in message_encode:
    state_ab = "10"
    qubits_ab = qubits.get_qubit_matrix(state_ab)
    qubit_c = qubits.get_qubit_matrix([m])
    qubit_b,state_ca = bell_state.teleportation(qubit_c,state_ab)
    
    qubit_b = bell_state.unitary_operation(qubit_b, 0, state_ca, state_ab)
    state_b = qubits.measurement(qubit_b,[0])[0]
    receive_buffer += state_b
    print("qubit_c(sender):{} | measurement_ca(send):{} | qubit_b(receiver):{}".format(m, state_ca, state_b))

qubit_c(sender):1 | measurement_ca(send):10 | qubit_b(receiver):1
qubit_c(sender):0 | measurement_ca(send):10 | qubit_b(receiver):0
qubit_c(sender):0 | measurement_ca(send):10 | qubit_b(receiver):0
qubit_c(sender):1 | measurement_ca(send):11 | qubit_b(receiver):1
qubit_c(sender):0 | measurement_ca(send):00 | qubit_b(receiver):0
qubit_c(sender):0 | measurement_ca(send):01 | qubit_b(receiver):0
qubit_c(sender):0 | measurement_ca(send):01 | qubit_b(receiver):0
qubit_c(sender):0 | measurement_ca(send):01 | qubit_b(receiver):0
qubit_c(sender):1 | measurement_ca(send):11 | qubit_b(receiver):1
qubit_c(sender):1 | measurement_ca(send):10 | qubit_b(receiver):1
qubit_c(sender):0 | measurement_ca(send):10 | qubit_b(receiver):0
qubit_c(sender):0 | measurement_ca(send):11 | qubit_b(receiver):0
qubit_c(sender):1 | measurement_ca(send):00 | qubit_b(receiver):1
qubit_c(sender):0 | measurement_ca(send):00 | qubit_b(receiver):0
qubit_c(sender):1 | measurement_ca(send):00 | qubit_b(receiver):1
qubit_c(se

In [6]:
print("[Receive]:{}".format(receive_buffer))
message_decode = bytes.fromhex(format(int(receive_buffer, base=2), 'x')).decode('utf-8')
print("[Decode]:{} ({} bianary bit)".format(message_decode,len(receive_buffer)))

[Receive]:10010000110010101101100011011000110111100100000010101110110111101110010011011000110010000100001
[Decode]:Hello World! (95 bianary bit)
